In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
np.random.seed(42)

import numpy as np
import pandas as pd
from numpy.linalg import lstsq, cholesky
from scipy.linalg import sqrtm, schur
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from ClassBSplines import BSpline
from TensorProductSplines import TensorProductSpline
from PenaltyMatrices import PenaltyMatrix
from Smooth import Smooths as s

from ClassBSplines import BSpline as b
from TensorProductSplines import TensorProductSpline as t

class Model():
    
    possible_penalties = { "smooth": PenaltyMatrix().D2_difference_matrix, 
                           "inc": PenaltyMatrix().D1_difference_matrix,
                           "dec": PenaltyMatrix().D1_difference_matrix,
                           "conc": PenaltyMatrix().D2_difference_matrix, 
                           "conv": PenaltyMatrix().D2_difference_matrix,
                           "peak": None }
    
    def __init__(self, descr, n_param=20):
        """
        descr : tuple - ever entry describens one part of 
                        the model, e.g.
                        descr =( (s(0), 10),
                                  (s(1), 15), 
                                  (t(0,1), 5) 
                               )
                        with the scheme: (type of smooth, number of knots)
        
        !!! currently only smooths with BSpline basis !!!
        """
        self.description_str = descr
        self.coef_ = None
        self.n_param = n_param
        self.description_dict = { t: (p, n) for t, p, n  in self.description_str}
        self.smooths = None

    def create_basis(self, X, n_param=10):
        """Create the unpenalized BSpline basis for the data X.
        
        Parameters:
        ------------
        X : np.ndarray - data
        n_param : int  - number of parameters for the spline basis 
        
        """
        if len(self.description_str) == X.shape[1]:
            pass
        else:
            print("Number of specified smooths must match number of columns in X!")
            print("Nr of smooths = ", len(self.description_str))
            print("Nr of columns in X = ", X.shape[1])
            return
        
        self.n_param = n_param       
        self.smooths = [ 
            s(x_data=X_norm[:, int(k[2])-1], 
            n_param=int(v[1]), 
            penalty=v[0]) for k, v in self.description_dict.items()
        ]    
        self.basis_without_penalty = np.concatenate([smooth.basis for smooth in self.smooths], axis=1) 
        
        return 
    
    def create_basis_with_penalty_and_weight(self, X):
        """Create the penalty matrices specified in self.description_str.
        
        Parameters:
        ---------------
        X : np.ndarray  - data
        
        TODO:
            [ ]  include the weights !!! 
        
        """
        
        pm = []
        if self.smooths is None:
            print("Run Model.create_basis() first!")
            return
            
        self.basis_with_penalty = np.concatenate(
            [np.vstack(
                (smooth.basis, smooth.penalty_matrix.T @ smooth.penalty_matrix)
            ) for smooth in M.smooths
            ], axis=1
        )
       
        return
        
    def fit(self, X, y, plot_=True, lam=None):
        """Lstsq fit try 2 using Smooths.
        
        Parameters:
        -------------
        X : pd.DataFrame or np.ndarray
        y : pd.DataFrame or np.array
        plot_ : boolean
        
        """
        print("Include one lambda per penalty!")
        if lam is None:
            lam = np.random.random(X.shape[1])
        else:
            print("Number of lambdas: ", len(lam))
        self.create_basis(X, n_param=self.n_param, lam=lam)
        if type(y) is pd.core.frame.DataFrame:
            y = y.values
                
        length_diff = int(self.basis_with_penalty.shape[0]) - y.shape[0]
        if length_diff == 0:
            y_fit = y
        else:
            #print("Shape y:", y.shape)
            #print("Shape padding: ", np.zeros((length_diff, 1)).shape)
            y_fit = np.append(y, np.zeros((length_diff, 1)))
          
        print(f"\n Shape of the finale basis with penalty: {self.basis_with_penalty.shape}")
        
        fitting = lstsq(a=self.basis_with_penalty, b=y_fit, rcond=None)
        self.coef_ = fitting[0]
        fitting = (*fitting, self.basis_without_penalty @ self.coef_)
    
        self.mse = mean_squared_error(y, fitting[-1])
        print(f"Mean squared error: {np.round(self.mse, 4)}")
        
        if plot_:
            fig = self.plot_xy(x=X[:,0], y=y.reshape((-1,)), name="Data")
            fig.add_trace(go.Scatter(x=X[:,0], y=fitting[-1], name="Fit", mode="markers"))
            fig.show()
        return fitting
    
    def predict(self, X, y=None, plot_=False):
        """Prediction of the trained model on the data in X."""
        if self.coef_ is None:
            print("Model untrained!")
            return
        
        self.create_basis(X, n_param=self.n_param, penalty=None)
        # prediction_basis, pad_y = self.multiple_smooths(X.values)
        #if pad_y is False:
        #    pass
        #else:
        #    prediction_basis = prediction_basis[:-len(pad_y)]
        
        print("Shape prediction basis: ", self.basis_without_penalty.shape)
        print("Shape coef_: ", self.coef_.shape)
        pred = self.basis_without_penalty @ self.coef_
        if plot_:
            fig = self.plot_xy(x=X[:,0], y=pred, name="Prediction")
            if type(y) is not None:
                print("shape y: ", y.shape)
                fig.add_trace(go.Scatter(x=X[:,0], y=y.reshape((-1,)), name="Data", mode="markers"))
            fig.show()
        return pred
    
    def plot_xy(self, x, y, title="Titel", name="Data", xlabel="xlabel", ylabel="ylabel"):
        """Basic plotting function."""
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, name=name, mode="markers"))
        fig.update_layout(title=title)
        fig.update_xaxes(title=xlabel)
        fig.update_yaxes(title=ylabel)
        return fig
    
    def plot_basis(self, matrix):
        """Plot the matrix."""
        fig = go.Figure(go.Image(z=matrix)).show()
        return
                        
      
    
def isSPD(matrix):
    try:
        np.linalg.cholesky(matrix)
        return True
    except np.linalg.LinAlgError as err:
        return False
    
def check_constraint(beta, constraint, print_idx=False):
    """Checks if beta fits the constraint."""
    V = np.zeros((len(beta), len(beta)))
    beta_diff = np.diff(beta)
    if constraint is "inc":
        v = [0 if i > 0 else 1 for i in beta_diff]
    elif constraint is "dec":
        v = [0 if i < 0 else 1 for i in beta_diff]
    V[:-1,:-1] = np.diag(v)
    if print_idx:
        print("Constraint violated at the followin indices: ")
        print([idx for idx, n in enumerate(v) if n == 1])
    return V
    
    
    

In [11]:
pm = []

m = ( ('s(1)', 'dec', 10), ('s(2)', 'inc', 10) ) # ('s(3)', 'smooth', 1), ('s(4)', 'smooth', 1) )
M = Model(m)
M.create_basis(X=X_norm)

        

px.imshow(basis_with_penalty)
#pm = [np.zeros((10,10)) if smooth.penalty is "no" else smooth.penalty_matrix]

Type x_data:  <class 'numpy.ndarray'>
Use 'x_basis' for the spline basis!
Type x_data:  <class 'numpy.ndarray'>
Use 'x_basis' for the spline basis!


In [2]:
#########################################################################
## DATA GENERATION
#########################################################################
np.random.seed(42)
X = pd.DataFrame(data={"x1": np.linspace(-2,2,500), "x2": np.linspace(0,1,500)
                      })#"x3": np.linspace(-2,2,500), "x4": np.linspace(0,1,500),
                       #"x5": np.linspace(-2,2,500), "x6": np.linspace(0,1,500)})
y = pd.DataFrame(data={"y": np.sin(X["x1"]) + 2*X["x2"] + np.random.randn(len(X["x1"]))*0.1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=True)

X_test = X_test.sort_index()
X_train = X_train.sort_index()
y_train = y_train.sort_index()
y_test = y_test.sort_index()

X_train.shape

X_norm = MinMaxScaler().fit_transform(X_train)
y_norm = StandardScaler().fit_transform(y_train)

In [3]:
m = ( ('s(1)', 'smooth', 10), ('s(2)', 'inc', 10) ) # ('s(3)', 'smooth', 1), ('s(4)', 'smooth', 1) )
M = Model(m)
M1 = Model(m)

M.create_basis(X=X_norm)
#px.imshow(M.basis_without_penalty)

M.smooths[1].penalty_matrix.shape


Type x_data:  <class 'numpy.ndarray'>
Use 'x_basis' for the spline basis!
Type x_data:  <class 'numpy.ndarray'>
Use 'x_basis' for the spline basis!


(9, 10)

In [22]:
#######################################################################
# trusted
#######################################################################
# BSpline functionality    
#######################################################################
def bSpline(x, k, i, m=2):
    """ 
    evaluate i-th b-spline basis function of order m at the 
    values in x, given knot loactions in k
    """
    if m==-1:
        # return 1 if x is in {k[i], k[i+1]}, otherwise 0
        return (x >= k[i]) & (x < k[i+1]).astype(int)
    else:
        #print("m = ", m, "\t i = ", i)
        z0 = (x - k[i]) / (k[i+m+1] - k[i])
        z1 = (k[i+m+2] - x) / (k[i+m+2] - k[i+1])
        return z0*bSpline(x, k, i, m-1) + z1*bSpline(x, k, i+1, m-1)

# trusted
def modelMatrixBSpline(x, k=0, m=2):
    """ 
    set up model matrix for the B-spline basis
    one needs k + m + 1 knots for a spline basis of order m with k 
    parameters,
    k - number of parameters (== number of B-splines)
    """
    n = len(x) # n = number of data
    assert (k > 0), "number of B-splines k need to be specified and larger than 0"
    xmin, xmax = np.min(x), np.max(x)
    xk = np.quantile(a=x, q=np.linspace(0,1,k))
    dx = xk[m+3] - xk[m+2]
    xk = np.insert(xk, 0, np.arange(xmin-(m+1)*dx, xmin, dx))
    xk = np.append(xk, np.arange(xmax+dx, xmax+(m+2)*dx, dx))
    X = np.zeros(shape=(n, k))
    for i in range(k):
        X[:,i] = bSpline(x=x, k=xk, i=i+1, m=m)
    return X, xk

#trusted
def pSplinePenalty(k):
    """ 
    compute the P-spline second order difference penalty matrix of 
    dimension k according to p.150 Wood 2006 
    """
    assert (type(k) is int), "Type of input k is not int"
    a =np.eye(k, dtype=np.int)
    P = np.diff(a, axis=0)
    S = P.T @ P
    return S

def plot_ModelMatrixBSpline(x, k, m=2):
    X, xk = modelMatrixBSpline(x, k=k, m=m)

    fig = go.Figure()
    for i in range(X.shape[1]):
        fig.add_trace(go.Scatter(x=x, y=X[:,i], 
                                 name=f"BSpline {i+1}", mode="lines"))
    for i in xk:
        addVertLinePlotly(fig, x0=i)
    fig.show()
    return

In [23]:
#def B_Splines
k, m = 10, 2
x = np.linspace(-100, 10,1000)

plot_ModelMatrixBSpline(x, k)

NameError: name 'go' is not defined

In [24]:
# trusted
def iterativeFit(x, y, k, type_="mi", lam_p=0.1, lam_c = 1e3, extremumIdx=None):
    # get B-Spline basis
    X, xk = modelMatrixBSpline(x, k=k)
    beta = np.zeros(X.shape[1])
    
    yExt = x[np.argmax(y)] if type_ is "peak" else x[np.argmin(y)]
    extremumIdx = int(np.argwhere(xk > yExt)[0]- 1)
    # print(f"Shape of model matrix: {X.shape}")
    # get P-Spline penalty matrix
    P_matrix = D1_differenceMatrix(k)
    # print(f"Shape of smooth-penalty matrix: {P_matrix.shape}")
    # get monotonicity penalty and weight matrix
    D1 = D1_differenceMatrix(k=k)    
    D2 = D2_differenceMatrix(k=k)
    
    if type_ in ["mi", "md"]:
        D = D1
    else:
        D = D2
        
    beta_hat = inv(X.T@X + lam_p * P_matrix.T @ P_matrix) @ X.T @ y
    y_hat = X @ beta_hat
    error = mse(y, y_hat)

    for i in range(3):
        V = V_weightMatrix(k=k, type_=type_, beta=beta_hat, extremumIdx=extremumIdx)
        xdot = X.T @ X
        pdot = P_matrix.T @ P_matrix
        ddot = D.T @ V @ D
        beta_hat = inv(xdot + lam_p * pdot + lam_c * ddot) @ X.T @ y
        y_hat = X @ beta_hat
        error = mse(y, y_hat)
        #print("Nonzeros in V: ", np.count_nonzero(np.diag(V)))
        #print(f"MSE: {np.round(error, 5)}")
    
    return beta_hat, y_hat

In [25]:
#######################################################################
# trusted
#######################################################################
# PENALTY MATRICES
#######################################################################
def V_weightMatrix(k=0, type_="mi", beta=None, extremumIdx=None):
    """
    calcualte the weight matrix for the different types of penalty,
    e.g. md, mi, convex, concave
    """
    notGivenError = " are of wrong type or not given!"
    if type_ is "mi":
        d = (np.diff(beta) < 0).astype(int)
        d = np.append(d, 0)
        addZeros = np.zeros(k)
    elif type_ is "md":
        d = (np.diff(beta) > 0).astype(int)
        d = np.append(d, 0)
        addZeros = np.zeros(k)
    elif type_ is "convex":
        assert (beta is not None), "beta"+notGivenError
        D2 = D2_differenceMatrix(k=k)
        d = (D2 @ beta < 0).astype(int)
    elif type_ is "concave":
        assert (beta is not None), "beta"+notGivenError
        D2 = D2_differenceMatrix(k=k)
        d = (D2 @ beta > 0).astype(int)
    elif type_ is "peak":
        assert (extremumIdx is not None), "extremumIdx"+notGivenError
        P = Peak_Matrix(k=k, peakIdx=extremumIdx)
        d = (P @ beta < 0).astype(int)
    elif type_ is "valley":
        assert (extremumIdx is not None), "extremumIdx"+notGivenError
        V = Valley_Matrix(k=k, valleyIdx=extremumIdx)
        d = (V @ beta < 0).astype(int)
    else:
        print(f"Requested type {type_} not implemented so far!")
        d = np.ones(k)
    
    offset = 0
    V = diags(d, offset, dtype=np.int).toarray()
    #print("Shape of V-Matrix: {}".format(V.shape))
   
    return V

# trusted
def Peak_Matrix(k=0, peakIdx=0):
    """ 
    calculate the peak penalty matrix for monotonic increasing
    and monotonic decreasing peak
    """
    assert (type(k) is int), "Type of input k is not int"
    assert (type(peakIdx) is int), "Type of input peakIdx is not int"

    dia = -1*np.ones(k)
    dia[peakIdx] = 0
    #dia[peakIdx-1:peakIdx+1] = 0
    dia[peakIdx+1:] = 1
    offdia = -1 * dia
    offset = [0,1]
    Peak = diags(np.array([dia, offdia]), offset, dtype=np.int).toarray()
    Peak[-1:] = 0
    return Peak

# trusted
def Valley_Matrix(k=0, valleyIdx=0):
    """ 
    calculate the peak penalty matrix for monotonic increasing
    and monotonic decreasing peak
    """
    assert (type(k) is int), "Type of input k is not int"
    assert (type(valleyIdx) is int), "Type of input peakIdx is not int"

    dia = 1*np.ones(k)
    dia[valleyIdx] = 0
    #dia[peakIdx-1:peakIdx+1] = 0
    dia[valleyIdx+1:] = -1
    offdia = -1 * dia
    offset = [0,1]
    Valley = diags(np.array([dia, offdia]), offset, dtype=np.int).toarray()
    Valley[-1:] = 0
    return Valley

# trusted
def D1_differenceMatrix(k=0):
    """
    calculated the first order difference matrix  
    returns a matrix of size [k x k-1]
    """
    assert (type(k) is int), "Type of input k is not int"
    d = np.array([-1*np.ones(k), np.ones(k)])
    offset=[0,1]
    D1 = diags(d,offset, dtype=np.int).toarray()
    D1[-1:] = 0.
    #print("Shape of D1-Matrix: {}".format(D1.shape))
    return D1

# trusted
def D2_differenceMatrix(k=0):
    """
    calculated the second order difference matrix  
    returns a matrix of size [k x k-1]
    """
    assert (type(k) is int), "Type of input k is not int"
    d = np.array([np.ones(k), -2*np.ones(k), np.ones(k)])
    offset=[0,1,2]
    D2 = diags(d,offset, dtype=np.int).toarray()
    D2[-2:] = 0.
    #print("Shape of D2-Matrix: {}".format(D2.shape))
    return D2

In [26]:
D1 = D1_differenceMatrix(k=10)
beta = np.random.rand(10)
print("Beta: \n", beta)
D1 @ beta

Beta: 
 [0.1191134  0.10683503 0.07902733 0.47567084 0.71116051 0.20377456
 0.24884892 0.64815352 0.98660911 0.93752108]


array([-0.01227837, -0.0278077 ,  0.39664351,  0.23548967, -0.50738595,
        0.04507436,  0.39930459,  0.33845559, -0.04908803,  0.        ])

In [27]:
constrDict = {"smooth": 0, "mi":0, "md":0, "convex":0, "concave":0}
def additiveModel_Matrix(x1, x2, k, y=None):
    """ 
    calculate the model matrix for P-splines for more than one
    dimension
    """
    x1, x2 = np.sort(x1), np.sort(x2)
    X1, X2 = modelMatrixBSpline(x1, k=k), modelMatrixBSpline(x2, k=k)
    X = np.hstack((X1[0], X2[0]))
    SP1, SP2 = sqrtm(pSplinePenalty(k=k)), sqrtm(pSplinePenalty(k=k))
    SP = np.block([[SP1, np.zeros((SP1.shape[0], SP2.shape[1]))],
                   [np.zeros((SP2.shape[0], SP1.shape[1])), SP2]])
    X = np.vstack((X, SP))
    
    if type(y) is None:
        return X.astype(np.float)
    else:
        y = np.append(y, np.zeros(2*k))
        return X.astype(np.float), y

def penalizedModelMatrix(x, y, k=0, b0=None):
    assert (b0 is not None), "Give initial guess for parameter vector beta!"
    penalties = np.array(
        [pSplinePenalty(k=k), 
                 D1_differenceMatrix(k=k).T @ V_weightMatrix(k, "mi", b0, None) @ D1_differenceMatrix(k),
                 D1_differenceMatrix(k=k).T @ V_weightMatrix(k, "md", b0, None) @ D1_differenceMatrix(k),
                 D2_differenceMatrix(k=k).T @ V_weightMatrix(k, "convex", b0, None) @ D2_differenceMatrix(k),
                 D2_differenceMatrix(k=k).T @ V_weightMatrix(k, "concave", b0, None) @ D2_differenceMatrix(k),
        ])
    lam = np.array(list(constrDict.values())).reshape((len(constrDict.values()),-1))
    p = [i * lam[c] for c, i in enumerate(penalties)]
    p2 = list(map(sqrtm, p))
    p3 = [p2[i].astype(np.int) if m==0 else p2[i].astype(np.float) 
          for i,m in enumerate(lam)]

    X = modelMatrixBSpline(x, k)[0]
    prod = reduce((lambda x1,x2: np.vstack((x1,x2))), p3)
    X_aug = np.vstack((X, prod))
    y_aug = np.append(y, np.zeros(int(len(penalties)*k)))
    
    return X_aug, y_aug

def fit_additiveModel_2d(x1, x2, y, k=1, plot_=False):
    """
    fit an additive model for 2 dimensions using P-Splines
    """
    X, y_aug = additiveModel_Matrix(x1=x1, x2=x2, k=k, y=y)
    beta_hat = lstsq(a=X, b=y_aug)[0]
    error = mse(y_aug, X @ beta_hat)
    print(f"Mean Squared Error: {error}")
    if plot_:
        fig = make_subplots(rows=2, cols=2,
                            specs=[[{}, {}], 
                                   [{"colspan": 2, "type":"scene"}, None]],)
        fig.add_trace(go.Scatter(x=x1, y=y, name="data", 
                                 mode="markers"), row=1, col=1)
        fig.add_trace(go.Scatter(x=x1, y=X@beta_hat, mode="markers+lines",
                                 name="Fit"), row=1, col=1)
        
        fig.add_trace(go.Scatter(x=x2, y=y, name="data", 
                                 mode="markers"), row=1, col=2)
        fig.add_trace(go.Scatter(x=x2, y=X@beta_hat, mode="markers+lines",
                                 name="Fit"), row=1, col=2)
        
        fig.add_trace(go.Scatter3d(x=x1, y=x2, z=y, name="data",
                                   mode="markers"), row=2, col=1)
        fig.add_trace(go.Scatter3d(x=x1, y=x2, z=X@beta_hat, name="fit", 
                                   mode="markers"), row=2, col=1)
        fig.update_traces(marker=dict(size=2))
        fig.show()
    return

def fit_additiveModel_2d_with_penalties(x1, x2, y, k=1, plot_=False):
    
    X1_pre = modelMatrixBSpline(x1, k=k)[0]
    X2_pre = modelMatrixBSpline(x2, k=k)[0]
    print("Shape of modelmatrix X1: ", X1_pre.shape)
    print("Shape of modelmatrix X2: ", X2_pre.shape)
    print("Shape of y: ", y.shape)
    b1_pre = lstsq(X1_pre, y)[0]
    b2_pre = lstsq(X2_pre, y)[0]
    
    X1, y1 = penalizedModelMatrix(x1, y, k=k, b0=b1_pre)
    X2, y2 = penalizedModelMatrix(x2, y, k=k, b0=b2_pre)
    X = np.hstack((X1, X2))
    print("Shape of the final model matrix: ", X.shape)
    print("Shape y1: ", y1.shape)
    print("Shape y2: ", y2.shape)
    y = y1 if np.sum(y1 - y2) < 1e-8 else print("Error")
    beta = lstsq(a=X, b=y)[0]
    
    if plot_:
        fig = make_subplots(rows=2, cols=2,
                            specs=[[{}, {}], 
                                   [{"colspan": 2, "type":"scene"}, None]],)
        fig.add_trace(go.Scatter(x=x1, y=y, name="data", 
                                 mode="markers"), row=1, col=1)
        fig.add_trace(go.Scatter(x=x1, y=X@beta, mode="markers+lines",
                                 name="Fit"), row=1, col=1)

        fig.add_trace(go.Scatter(x=x2, y=y, name="data", 
                                 mode="markers"), row=1, col=2)
        fig.add_trace(go.Scatter(x=x2, y=X@beta, mode="markers+lines",
                                 name="Fit"), row=1, col=2)

        fig.add_trace(go.Scatter3d(x=x1, y=x2, z=y, name="data",
                                   mode="markers"), row=2, col=1)
        fig.add_trace(go.Scatter3d(x=x1, y=x2, z=X@beta, name="fit", 
                                   mode="markers"), row=2, col=1)
        fig.update_traces(marker=dict(size=2))
        fig.show()
    
    return beta

In [28]:
#x, y = f2(xmin=-3, xmax=3, noiseType="nope")
x = np.linspace(0,9,1000)
y = f3(x) + 0.5 * np.random.randn(len(x))

beta_hat, y_hat = iterativeFit(x, y, k=24, type_="mi", lam_p=1, lam_c=0)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=x, y=y, name="data", mode="markers"), row=1, col=1)
fig.add_trace(go.Scatter(x=x, y=y_hat, name="Fit"), row=1, col=1)
fig.add_trace(go.Scatter(x=x[:-1], y=np.diff(y_hat), name="Differenz in y"), row=2, col=1)
fig.update_yaxes(title_text="", type="log", row=2, col=1)

NameError: name 'f3' is not defined

In [29]:
def fit_1d(x, y, k=1, smooth=True, plot_=False):
    """
    """
    if smooth:
        beta_init = np.random.random(k)
        Xa, ya = penalizedModelMatrix(x, y, k=k, b0=beta_init)
    else:
        Xa, xk = modelMatrixBSpline(x=x, k=k)
        ya = y
    #print("Shape Xa: ", Xa.shape)
    #print("Shape ya: ", ya.shape)
    beta_hat = lstsq(a=Xa, b=ya)[0]
    error = mse(ya, Xa @ beta_hat)
    print(f"Mean Squared Error: {error}")
    if plot_:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x1, y=y, name="data", 
                                 mode="markers"))
        fig.add_trace(go.Scatter(x=x1, y=Xa@beta_hat, mode="lines",
                                 name="Fit"))
        fig.update_traces(marker=dict(size=2))
        fig.show()
    return beta_hat, Xa

In [30]:
ne = 10
k = 6
x = np.linspace(0,3,ne)
y = np.linspace(0,5,ne)
xg, yg = np.meshgrid(x, y)
X = modelMatrixBSpline(x, k)[0]
Y = modelMatrixBSpline(y, k)[0]

K = kron(X,Y).toarray()
print("X: ", xg.shape)
print("Y :", yg.shape)
print("K :", K.shape)
fig = go.Figure()
for i in range(K.shape[1]):
    fig.add_trace(go.Surface(x=xg, y=yg, z=K[:,i].reshape((ne, ne))))
fig.show()

X:  (10, 10)
Y : (10, 10)
K : (100, 36)


NameError: name 'go' is not defined

In [31]:
!pip install ndsplines

  Using cached ndsplines-0.1.1-cp37-cp37m-win_amd64.whl (84 kB)


In [32]:
!pip install cython

In [33]:
# Test out ndsplines 0.1.1 
# https://pypi.org/project/ndsplines/

import ndsplines
import numpy as np
from scipy import interpolate
import numpy as np

NUM_X = 10
NUM_Y = 10
x = np.linspace(-3, 3, NUM_X)
y = np.linspace(-3, 3, NUM_Y)
meshx, meshy = np.meshgrid(x,y, indexing='ij')
input_coords = np.stack((meshx, meshy), axis=-1)
K = np.array([[1, -0.7,], [-0.7, 1.5]])
meshz = np.exp(-np.einsum(K, [1,2,], input_coords, [...,1], input_coords, [...,2])) + 0.1 * np.random.randn(NUM_X,NUM_Y)
#print("meshx: \n", meshx)
#print("meshy: \n", meshy)
#print("input_coords: \n", input_coords)
xt = [-1, 0, 1]
yt = [-1, 0, 1]
k = 3
xt = np.r_[(x[0],)*(k+1), xt, (x[-1],)*(k+1)]
yt = np.r_[(y[0],)*(k+1), yt, (y[-1],)*(k+1)]
ts = [xt, yt]

samplex = input_coords.reshape((-1,2))
sampley = meshz.reshape((-1))

spl = ndsplines.make_lsq_spline(samplex, sampley, ts, np.array([3,3]))

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=meshx.flatten(), 
                           y=meshy.flatten(), 
                           z=meshz.flatten(), mode="markers", name="data"))
fig.add_trace(go.Surface(x=meshx, y=meshy, z=spl(input_coords), name="fit"))
fig.update_traces(marker=dict(size=4), selector=dict(mode="markers"))
fig.show()

NameError: name 'go' is not defined